In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
class Gripper:
    def __init__(self, len_x, len_y, len_z):
        self._closure_volume_size = np.array([len_x, len_y, len_z])
        self._transforms = self._create_transforms(finger_size=[0.01, len_y, len_z], palm_size=[len_x, len_y, 0.01])
        self._meshes = self._create_meshes(finger_size=[0.01, len_y, len_z], palm_size=[len_x, len_y, 0.01])

    def _create_transforms(self, finger_size, palm_size):
        transforms = {}
        transforms['tip_palm'] = np.eye(4)
        transforms['tip_right_finger'] = np.eye(4)
        transforms['tip_left_finger'] = np.eye(4)
        transforms['closure_volume'] = np.eye(4)

        # Palm transform
        transforms['tip_palm'][0, 3] = -palm_size[0] / 2
        transforms['tip_palm'][1, 3] = -palm_size[1] / 2
        transforms['tip_palm'][2, 3] = -finger_size[2] - palm_size[2]

        # Right finger transform
        transforms['tip_right_finger'][0, 3] = palm_size[0] / 2 - finger_size[0]
        transforms['tip_right_finger'][1, 3] = -palm_size[1] / 2
        transforms['tip_right_finger'][2, 3] = -finger_size[2]

        # Left finger transform
        transforms['tip_left_finger'][0, 3] = -palm_size[0] / 2
        transforms['tip_left_finger'][1, 3] = -palm_size[1] / 2
        transforms['tip_left_finger'][2, 3] = -finger_size[2]

        # Closure volume transform
        transforms['closure_volume'][0, 3] = -palm_size[0] / 2 + finger_size[0]
        transforms['closure_volume'][1, 3] = -palm_size[1]
        transforms['closure_volume'][2, 3] = -finger_size[2]

        return transforms
    
    def check_collision(self, points: o3d.geometry.PointCloud, threshold: float = 0.01) -> bool:
        """
        Check if the points in the point cloud are within the closure volume of the gripper.
        """
        closure_volume = self._transforms['closure_volume']
        closure_volume_size = self._closure_volume_size
        
        # Calculate the bounds of the closure volume
        min_bounds = closure_volume[:3, 3] - closure_volume_size / 2
        max_bounds = closure_volume[:3, 3] + closure_volume_size / 2
        
        # Check if any point is within the bounds
        for point in points.points:
            if (min_bounds <= point).all() and (point <= max_bounds).all():
                return True
        return False

    def _create_meshes(self, finger_size, palm_size):
        meshes = {}
        
        right_finger_mesh = o3d.geometry.TriangleMesh.create_box(finger_size[0], finger_size[1], finger_size[2], True)
        right_finger_mesh.translate(self._transforms['tip_right_finger'][:3, 3])
        right_finger_mesh.paint_uniform_color([0.1, 0.4, 0.8])
        meshes['right_finger'] = right_finger_mesh

        left_finger_mesh = o3d.geometry.TriangleMesh.create_box(finger_size[0], finger_size[1], finger_size[2])
        left_finger_mesh.translate(self._transforms['tip_left_finger'][:3, 3])
        left_finger_mesh.paint_uniform_color([0.5, 0.1, 0.6])
        meshes['left_finger'] = left_finger_mesh

        palm_mesh = o3d.geometry.TriangleMesh.create_box(palm_size[0], palm_size[1], palm_size[2])
        palm_mesh.translate(self._transforms['tip_palm'][:3, 3])
        palm_mesh.paint_uniform_color([0.4, 0.1, 0.3])
        meshes['palm'] = palm_mesh

        frame_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()
        frame_mesh = frame_mesh.scale(0.1, center=[0,0,0])
        meshes['frame'] = frame_mesh

        return meshes

    @property
    def meshes(self):
        return list(self._meshes.values())


In [19]:
gripper = Gripper(0.05, 0.03, 0.05)
meshes = gripper.meshes
o3d.visualization.draw_geometries(meshes)

In [ ]:
len(meshes)

4